In [ ]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.6 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

In [ ]:
# Loading  pre-trained asafaya/bert-base-arabic model and tokenizer

tokenizer = BertTokenizer.from_pretrained("asafaya/bert-base-arabic")

model = BertForQuestionAnswering.from_pretrained("asafaya/bert-base-arabic")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at asafaya/bert-base-arabic and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data = "الغسالة تعتبر جهازًا أساسيًا في أي منزل، حيث توفر وسيلة سهلة وفعالة لغسل الملابس. تتوفر الغسالات بمجموعة متنوعة من الأحجام والميزات، بما في ذلك البرامج المخصصة لأنواع معينة من الأقمشة ودرجات الاتساخ."


In [ ]:
# data = "لغسالة هي جهاز كهربائي يستخدم لغسل الملابس. يمكن استخدامها لغسل مختلف أنواع الملابس بما في ذلك القمصان والسراويل والفساتين. تتميز الغسالات الحديثة بعدة وظائف مثل الغسيل السريع والتجفيف الذاتيh."


In [ ]:
#data = "لاستخدام الغسالة بشكل صحيح، يجب اتباع بعض الخطوات الأساسية. أولاً، قم بفتح الغطاء ووضع الملابس بداخل الغسالة بحذر، مع الأخذ في الاعتبار الحد الأقصى للحمولة. ثانياً، أضف المنظف المناسب وحدد الإعدادات المناسبة وفقًا لنوع الملابس ودرجة الاتساخ. ثالثاً، اضغط على الزر لبدء دورة الغسيل واختر الوقت المناسب. رابعًا، بعد الانتهاء، قم بإخراج الملابس ونقها بالماء النظيف واتركها لتجف. وأخيراً، قم بإيقاف تشغيل الغسالة وإغلاق الغطاء بإحكام. تأكد من اتباع تعليمات الصانع والحفاظ على صيانة دورية للحفاظ على أداء الغسالة بأفضل حال."

In [ ]:
question = input(":اكتب سؤالك هنا  ")


:اكتب سؤالك هنا  ما هي وظيفة الغسالة؟


In [ ]:
#              ما هي الخطوات الأساسية لاستخدام الغسالة بشكل صحيح؟
 #                    ما هي وظيفة الغسالة في المنزل؟

#          ما هي الإعدادات التي يجب اختيارها عند استخدام الغسالة وفقًا لنوع الملابس؟


In [ ]:
def get_answer(question, data, model, tokenizer):
    inputs = tokenizer(question, data, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
    return answer


In [ ]:
def print_answer(question, answer):
    print(f"سؤال: {question}")
    print(f"جواب: {answer}")


In [ ]:

result = get_answer(question, data, model, tokenizer)
print_answer(question, result)


سؤال: ما هي وظيفة الغسالة؟
جواب: متنوعة من الاحجام والميزات ، بما في ذلك البرامج المخصصة لانواع معينة من الاقمشة


In [ ]:
#---------------------------------------------

In [ ]:
# fine tuning

In [29]:
import torch
from transformers import BertTokenizer, BertForMaskedLM, AdamW, get_linear_schedule_with_warmup


In [30]:
file_path = "/content/data.txt"  # Replace with the path to your text file
with open(file_path, 'r', encoding='utf-8') as file:
    data = file.read()

In [31]:
tokenizer = BertTokenizer.from_pretrained("asafaya/bert-base-arabic")
input_text = "ظبت مؤقت المطبخ"

In [32]:
inputs = tokenizer(input_text, padding="max_length", max_length=512, truncation=True, return_tensors="pt")


In [33]:
model = BertForMaskedLM.from_pretrained("asafaya/bert-base-arabic")


Some weights of the model checkpoint at asafaya/bert-base-arabic were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
optimizer = AdamW(model.parameters(), lr=5e-5)
epochs = 3
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=epochs)

In [35]:
for epoch in range(epochs):
    outputs = model(**inputs)
    loss = outputs.loss
    if loss is not None:
        loss.backward()
        optimizer.step()
        scheduler.step()
        model.zero_grad()
    else:
        print("Loss is None. Skipping this batch.")

Loss is None. Skipping this batch.
Loss is None. Skipping this batch.
Loss is None. Skipping this batch.


In [36]:
def generate_answer(text, model, tokenizer, max_length=512):
    inputs = tokenizer(text, padding="max_length", max_length=max_length, truncation=True, return_tensors="pt")
    generated_ids = model.generate(input_ids=inputs.input_ids[:, :max_length], max_length=max_length, num_beams=5, early_stopping=True)
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [37]:
input_text = "ظبت مؤقت المطبخ؟"
result = generate_answer(input_text, model, tokenizer)
print(f"Input Text: {input_text}")
print(f"Generated Answer: {result}")

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 512, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


RuntimeError: ignored